This notebook shows a basic example how to generate a SPARQL query from an end user input.<br>
We use a LLM and a KG ontology to guide the query generation.

Requirements:
* pandas
* hana-ml (https://pypi.org/project/hana-ml/)
* gen-ai-hub-sdk (https://pypi.org/project/generative-ai-hub-sdk/)
* You loaded the "wiki_movies" graph and the corresponding ontology "wiki_movies_ontology"

1. Importing Libraries and Setting Up Environment Variables

In [1]:
import hana_ml
from hdbcli import dbapi
print(hana_ml.__version__)

2.24.25040300


2. Loading Environment Variables & Connecting to SAP HANA


In [ ]:

from hana_ml.dataframe import ConnectionContext
from dotenv import load_dotenv, set_key
import os

# Load environment variables from .env file
load_dotenv('.env')

# Retrieve variables from .env file
address = os.getenv('HANA_ADDRESS')
port = os.getenv('HANA_PORT')
user = os.getenv('HANA_USER')
password = os.getenv('HANA_PASSWORD')
encrypt = os.getenv('HANA_ENCRYPT') == 'True'
ssl_validate_certificate = os.getenv('HANA_SSL_VALIDATE_CERTIFICATE') == 'True'

# Connect to HANA using hana-ml
cc = ConnectionContext(
    address=address,
    port=port,
    user=user,
    password=password,
    encrypt=encrypt,
    sslValidateCertificate=ssl_validate_certificate,
)

print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1743496780 (fa/CE2025.2)
DBADMIN


4. Setting Default Headers and Prefixes for SPARQL Queries


In [5]:
# set the default headers for the SPARQL_EXECUTE procedure and the default prefixes for the SPARQL queries
rqx_hdrs = '''Accept: application/sparql-results+csv\r\nContent-Type: application/sparql-query'''
prefixes = '''
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX demo: <http://kg.demo.sap.com/>
'''

5. Executing SPARQL Queries

In [7]:
import pandas as pd
import io

def call_sparql_execute(qrystr:str, ctype):
        '''execute_query() - executes SPARQL query
             Returns a tuple of (content-type, responsedata) to be send back
        '''
        if ctype is None: ctype = 'application/sparql-results+json'
        
        if qrystr is None: return (ctype, '')
        rqx_hdrs = ''
        sep = '\r\n'
        rqx_hdrs += 'Accept: ' + ctype + sep
        rqx_hdrs += 'Content-Type: application/sparql-query' + sep
        cursor = cc.connection.cursor()
        try:
            r = cursor.callproc('SYS.SPARQL_EXECUTE', (qrystr, rqx_hdrs, '?', None))
            # print(qrystr, rqx_hdrs)
            resp = r[2]
            csize = -1
            for rh in r[3].split('\n'):
                hv = rh.split(':', 1)
                if len(hv) != 2: continue
                nm = hv[0].strip().lower()
                val = hv[1].strip()
                if nm == 'content-type': ctype = val
                elif nm == 'content-size': csize = int(val)
            if 0 == csize:
                ctype = 'text/plain'
        except dbapi.Error as dberr:
            resp = dberr.errortext.split('; Server Connection',1)[0]
        cursor.close()
        return (ctype, resp)

def convert_to_dataframe(result):
    result_df = pd.read_csv(io.StringIO(result))
    return result_df.fillna('')

6. Retrieving Ontology
</br>
</br> Retrieve Ontology: Execute a SPARQL query to retrieve the ontology data.



In [8]:
# first, we get the "wiki_movies_ontology" which we then use in the LLM prompt to generate valid SPARQL queries
sparql = prefixes + '''
CONSTRUCT
FROM <wiki_movies_ontology>
WHERE {?s ?p ?o}
'''

_, ontology = call_sparql_execute(sparql, rqx_hdrs)
print(ontology)

@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
<http://kg.demo.sap.com/Genre> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://kg.demo.sap.com/Director> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://kg.demo.sap.com/Film> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://kg.demo.sap.com/acted_in> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty> .
<http://kg.demo.sap.com/CastMemberStatement> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://kg.demo.sap.com/Person> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.w3.org/2000/01/rdf-schema#label> <http://www.w3.org/2000/01/rdf-schema#domain> <http://kg.demo.sap.com/Place> .
<http://www.w3.org/2000/01/rdf-schema#label> <http://www.w3.org/2000/01

7. Using Generative AI to Create SPARQL Queries
</BR> Define Function: Define a function to interact with a generative AI model to create SPARQL queries based on user input.
</BR> Generate Query: Use the AI model to generate a SPARQL query, retrieve literals, triples, and follow-up prompts.
</BR> Execute Query: Execute the generated SPARQL query and convert the results to a DataFrame for display.



In [9]:
# use a LLM from gen AI hub sdk to run the prompt
from gen_ai_hub.proxy.native.openai import chat
import json

def run(user_msg):
    sys_msg = f'''Given the ontology below, create a SPARQL query from the user prompt.
    Enclose literals in double quotes. Note that the graph is directed. Edges go from the domain to the range.
    If an rdf label exists for a class, always retrieve the label.
    In addition, list all the literals used in the SPARQL query along with a complete list of triples in the SPARQL query in which the identified literals occur.
    In addition, generate 5 potential follow-up prompts that extend the initial prompt in a meaningful way.
    Return a valid json only without code block in the following format:
    {{"data":{{"sparql_query": sparql query, "literals": [{{"literal": the literal}}], "triples_with_literals": [{{"triple": the triple with literal}}], "followup_prompts":[{{"prompt": the follow-up prompt}}]}}}}
    <ontology>
    {ontology}
    </ontology>'''
    
    messages=[{"role": "system", "content": sys_msg}, {"role": "user", "content": user_msg}]

    kwargs = dict(model_name='gpt-4o', messages=messages, response_format={ "type": "json_object" }, temperature=0)

    response = chat.completions.create(**kwargs)  

    data = json.loads(response.choices[0].message.content)
    print('user input:', user_msg, '\n')
    print('generated query:\n', data['data']['sparql_query'])
    print(data['data']['literals'])
    print(data['data']['triples_with_literals'])
    print(data['data']['followup_prompts'])

    _, r = call_sparql_execute(data['data']['sparql_query'], rqx_hdrs)

    s_df = convert_to_dataframe(r)
    display(s_df)

user_msg = '''Find all film titles directed by Denis Villeneuve and their genre'''

run(user_msg)

user input: Find all film titles directed by Denis Villeneuve and their genre 

generated query:
 SELECT ?filmTitle ?genreLabel WHERE { ?director a <http://kg.demo.sap.com/Director> ; <http://www.w3.org/2000/01/rdf-schema#label> "Denis Villeneuve" . ?director <http://kg.demo.sap.com/directed> ?film . ?film <http://kg.demo.sap.com/title> ?filmTitle . ?film <http://kg.demo.sap.com/genre> ?genre . ?genre <http://www.w3.org/2000/01/rdf-schema#label> ?genreLabel . }
[{'literal': 'Denis Villeneuve'}]
[{'triple': '?director <http://www.w3.org/2000/01/rdf-schema#label> "Denis Villeneuve" .'}]
[{'prompt': 'List all actors who acted in films directed by Denis Villeneuve.'}, {'prompt': 'Find the publication dates of films directed by Denis Villeneuve.'}, {'prompt': 'Retrieve the character roles in films directed by Denis Villeneuve.'}, {'prompt': "Get the places of birth of actors in Denis Villeneuve's films."}, {'prompt': 'List all directors who have directed films in the same genre as Denis Vil

,filmTitle,genreLabel
0,Dune: Part Two,film based on a novel
1,Dune: Part Two,science fiction film
2,Dune: Part Two,action film
3,Dune: Part Two,adventure film
4,Dune: Part Two,speculative fiction film
5,Dune: Part Two,epic film
6,Dune,film based on a novel
7,Dune,science fiction film
8,Dune,action film
9,Dune,adventure film


In [13]:

user_msg = '''List film titles and the date in which they were published'''
user_msg = '''List film titles and their directors'''
# user_msg = '''which films have more than 1 director'''
# user_msg = '''List all directors for each film that has more than one director.'''
# user_msg = '''provide a concatenated List all directors for each film that has more than one director.'''
# user_msg = '''Retrieve a count of directors for each film, displaying only those film titles with more than two directors.'''
# user_msg = '''Retrieve the names of directors for each film that has exactly three directors.'''

# user_msg = '''retrieve actor names and director names who they worked with'''
# user_msg = '''retrieve actor names who worked with Denis Villeneuve'''

# user_msg = '''List films and their directors'''
# user_msg = '''List films directed by Denis Villeneuve'''
# user_msg = '''Find all film titles directed by Denis Villeneuve and their genre'''
# user_msg = '''Find all film titles directed by Denis Villeneuve and a concatenated list of genres'''

# user_msg = '''List film title in which Zendaya acted'''
# user_msg = '''List directors who worked with Zendaya'''

# user_msg = '''list actors and their roles of Dune: Part Two'''
# user_msg = '''who played Chani in Dune: Part Two'''
# user_msg = '''What other film title has the actor who played Chani been in?''' 

# user_msg = '''Which actors played the most roles?'''
# user_msg = '''Which role in which movie did Lashana Lynch play?'''

# user_msg = '''Which actors colaborated in the most movies?'''
# user_msg = '''In which movies did Lashana Lynch and Kingsley Ben-Adir work together?'''

# user_msg = '''In which movies did Lashana Lynch and Kingsley Ben-Adir work together?'''

# user_msg = '''What do you knwo about Bob Marley: One Love?'''#fails

# user_msg = '''Which persons have been involved in Bob Marley: One Love? for actors, provide also the birth location if known'''

# user_msg = '''Who acted as Chani and when was she born?'''
# user_msg = '''Which actors were born before 2000?'''
user_msg = '''Which actors were less than 20 years old when they played in a movie? provide the movie title'''
# user_msg = '''list actors who were less than 20 years old when they played in a movie? provide also the movie title, publication date and age of actor'''

user_msg = '''which actor was born in Syndey'''
user_msg = '''which director also played a role in his film. also list the movie titles and the role they played'''

run(user_msg)

user input: which director also played a role in his film. also list the movie titles and the role they played 

generated query:
 SELECT ?directorLabel ?filmTitle ?characterRole WHERE { ?director a <http://kg.demo.sap.com/Director> . ?director <http://www.w3.org/2000/01/rdf-schema#label> ?directorLabel . ?director <http://kg.demo.sap.com/directed> ?film . ?film <http://kg.demo.sap.com/title> ?filmTitle . ?film <http://kg.demo.sap.com/castMemberStatement> ?castMemberStatement . ?castMemberStatement <http://kg.demo.sap.com/castMember> ?director . ?castMemberStatement <http://kg.demo.sap.com/characterRole> ?characterRole . }
[{'literal': 'Film a director directed'}, {'literal': 'Title of the film'}, {'literal': 'Cast member'}, {'literal': 'Character role'}]
[{'triple': '?director <http://kg.demo.sap.com/directed> ?film .'}, {'triple': '?film <http://kg.demo.sap.com/title> ?filmTitle .'}, {'triple': '?castMemberStatement <http://kg.demo.sap.com/castMember> ?director .'}, {'triple': '?cast

,directorLabel,filmTitle,characterRole
0,Sam Hargrave,Extraction II,Ditch Digger
1,David Leitch,Bullet Train,Jeff Zufelt
2,Bradley Cooper,Maestro,Leonard Bernstein
3,Patrick Wilson,Insidious: The Red Door,Josh Lambert
4,Michael B. Jordan,Creed III,Adonis Creed
5,Kenneth Branagh,Death on the Nile,Hercule Poirot
6,Kenneth Branagh,A Haunting in Venice,Hercule Poirot
7,Matthias Schweighöfer,L'Armée des voleurs,Ludwig Dieter / Sebastian Schlencht-Wöhnert
8,Matthias Schweighöfer,Army of Thieves,Ludwig Dieter / Sebastian Schlencht-Wöhnert
9,Jake Johnson,Self Reliance,Tommy
